<a href="https://colab.research.google.com/github/rajat0304/SIH-full-stack-website-with-integration-of-AI-/blob/main/repo01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!mkdir -p /content/praxics_meditech
%cd /content/praxics_meditech


/content/praxics_meditech


In [3]:
!pip install fastapi uvicorn python-multipart
!pip install numpy pandas matplotlib scikit-learn
!pip install tensorflow torch torchvision
!pip install pydantic
!pip install kaggle


In [5]:
import kagglehub

# to train model for xray.
path = kagglehub.dataset_download("orvile/bone-fracture-dataset")

print("Path to dataset files:", path)


Using Colab cache for faster access to the 'bone-fracture-dataset' dataset.
Path to dataset files: /kaggle/input/bone-fracture-dataset


In [6]:
import os

data_dir = path
print(os.listdir(data_dir))


['bone.txt', 'Bone fracture dataset']


In [13]:
import os

base_path = "/content/reposcan"
xray_path = "/content/reposcan/xray_module"

os.makedirs(xray_path, exist_ok=True)
print("Folders Created:", os.listdir("/content"))


Folders Created: ['.config', 'reposcan', 'praxics_meditech', 'sample_data']


In [14]:
import shutil

src_path = path      # KaggleHub dataset path
dst_path = "/content/reposcan/xray_module/dataset"

shutil.copytree(src_path, dst_path, dirs_exist_ok=True)

print("Dataset Copied To:", dst_path)
print("Files:", os.listdir(dst_path))


Dataset Copied To: /content/reposcan/xray_module/dataset
Files: ['bone.txt', 'Bone fracture dataset']


In [15]:
print(os.listdir(dst_path))


['bone.txt', 'Bone fracture dataset']


In [16]:
inner_path = "/content/reposcan/xray_module/dataset/Bone fracture dataset"
import os
print(os.listdir(inner_path))


['Bone fracture dataset']


In [17]:
print(os.listdir(inner_path))


['Bone fracture dataset']


In [18]:
import os

level2 = "/content/reposcan/xray_module/dataset/Bone fracture dataset/Bone fracture dataset"
print(os.listdir(level2))


['Dataset']


In [19]:
print(os.listdir(level2))


['Dataset']


In [20]:
import os

final_path = "/content/reposcan/xray_module/dataset/Bone fracture dataset/Bone fracture dataset/Dataset"
print(os.listdir(final_path))


['normal', 'fracture']


In [22]:
print(os.listdir(final_path))# to check the path of the folder


['normal', 'fracture']


In [23]:
# desinging the CNN model
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint

final_path = "/content/reposcan/xray_module/dataset/Bone fracture dataset/Bone fracture dataset/Dataset"


In [24]:
#to spilt the data and validation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=10,
    zoom_range=0.1,
    horizontal_flip=True
)

train = train_datagen.flow_from_directory(
    final_path,
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary',
    subset='training'
)

val = train_datagen.flow_from_directory(
    final_path,
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary',
    subset='validation'
)


Found 1702 images belonging to 2 classes.
Found 425 images belonging to 2 classes.


In [25]:
#model architecture and transfer of learning
base = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224,224,3))
base.trainable = False   # freeze base model

x = base.output
x = Flatten()(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.3)(x)
output = Dense(1, activation="sigmoid")(x)

model = Model(inputs=base.input, outputs=output)
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

model.summary()


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 10,286,401 (39.24 MB)

 Trainable params: 8,028,417 (30.63 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [26]:
#to train the model on the server
checkpoint = ModelCheckpoint("xray_fracture_model.h5", save_best_only=True, monitor="val_accuracy", mode="max")

history = model.fit(
    train,
    validation_data=val,
    epochs=10,
    callbacks=[checkpoint]
)
# xray datatraining ends here

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 337ms/step - accuracy: 0.8881 - loss: 4.1971

107/107 ━━━━━━━━━━━━━━━━━━━━ 76s 546ms/step - accuracy: 0.8884 - loss: 4.1804 - val_accuracy: 0.9459 - val_loss: 0.8684
Epoch 2/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 232ms/step - accuracy: 0.9657 - loss: 0.2015

107/107 ━━━━━━━━━━━━━━━━━━━━ 32s 299ms/step - accuracy: 0.9658 - loss: 0.2007 - val_accuracy: 0.9765 - val_loss: 0.1632
Epoch 3/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 32s 302ms/step - accuracy: 0.9892 - loss: 0.0423 - val_accuracy: 0.9718 - val_loss: 0.1248
Epoch 4/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 32s 303ms/step - accuracy: 0.9914 - loss: 0.0192 - val_accuracy: 0.9624 - val_loss: 0.2224
Epoch 5/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 232ms/step - accuracy: 0.9868 - loss: 0.0567

107/107 ━━━━━━━━━━━━━━━━━━━━ 32s 297ms/step - accuracy: 0.9868 - loss: 0.0567 - val_accuracy: 0.9953 - val_loss: 0.0240
Epoch 6/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 32s 302ms/step - accuracy: 0.9957 - loss: 0.0139 - val_accuracy: 0.9812 - val_loss: 0.0891
Epoch 7/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 32s 296ms/step - accuracy: 0.9939 - loss: 0.0298 - val_accuracy: 0.9576 - val_loss: 0.3297
Epoch 8/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 32s 297ms/step - accuracy: 0.9823 - loss: 0.0930 - val_accuracy: 0.9765 - val_loss: 0.0719
Epoch 9/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 32s 302ms/step - accuracy: 0.9907 - loss: 0.0298 - val_accuracy: 0.9576 - val_loss: 0.3651
Epoch 10/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 31s 293ms/step - accuracy: 0.9896 - loss: 0.0604 - val_accuracy: 0.9882 - val_loss: 0.1061


In [103]:
#here the UI desing start
!pip install gradio


In [110]:
import gradio as gr
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

MODEL_PATH = "/content/praxics_meditech/xray_fracture_model.h5"
model = load_model(MODEL_PATH)

def predict_xray(img):
    img = img.convert("RGB")  # FIX 1: Force RGB
    img = img.resize((224, 224))

    img_arr = image.img_to_array(img) / 255.0
    img_arr = np.expand_dims(img_arr, axis=0)

    pred = model.predict(img_arr)[0][0]

    # FIX 2: Lower threshold
    if pred > 0.40:
        return f"no issue found (Conf: {pred:.2f})"
    else:
        return f"fractured bone detected (Conf: {(1 - pred):.2f})"

demo = gr.Interface(fn=predict_xray, inputs=gr.Image(type="pil"), outputs="text")
demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://035105b241326170dc.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
